In [1]:
import sys
import string
from scipy import sparse
import numpy as np
import ipywidgets as widgets
import pandas as pd

from html import escape
from IPython.core.display import display, HTML
from string import Template
import json

HTML('<script src="https://d3js.org/d3.v3.min.js" charset="utf-8"></script>')

In [2]:
local_json_filepath = "\"Raw_HTML_And_Data/example_test_result.json\""
local_json = pd.read_json("Raw_HTML_And_Data/example_test_result.json")
local_json.head()

,index,true_label,predicted_label,confidence_score,text
0,0,1,1,"[0.574, 0.088, 0.036, 0.088, 0.214]",us prepares for hybrid onslaught sales of hybr...
1,1,2,1,"[0.4, 0.153, 0.168, 0.145, 0.134]",research fears over kelly views scientists hav...
2,2,2,2,"[0.103, 0.172, 0.397, 0.133, 0.196]",peace demo appeal rejected peace protest ##ors...
3,3,1,2,"[0.347, 0.1, 0.36, 0.09, 0.103]",salary scandal in cameroon cameroon says wides...
4,4,1,1,"[0.385, 0.159, 0.127, 0.216, 0.113]",eu ministers to mu ##ll jet fuel tax european ...


In [3]:
css_text = '''
    h1{
        color: white;
    }
    .dispcont {
        background-color: black;
        font-family: Arial, sans-serif;
        font-size: larger;
    }
    .box_highlighted { 
        background-color: #ffb; 
        border: 1px solid #b53;
    }

    .highlight{
        background-color: yellow;
    }

    .lighthigh{
        background-color: green;
    }

    li{
        font-size: smaller;
    }

    td{
        min-width: 100px;
    }

    #review{
        border:1px solid pink; 
        padding: 5px; 
        float: right; 
        width: 800px; 
        height: 1100px; 
        background-color: white;
        color: black;
        margin: 20px;
        overflow: scroll;
    }

    #matrix{
        border:1px solid pink; 
        padding: 5px; 
        float: left;
        width: 1200px; 
        height: 1100px;
        margin: 20px;
        overflow: scroll;
    }
'''

In [4]:
js_template = Template('''
console.log("Loading JavaScript...")
var conf_data = $confusion_data

console.log("Loaded Data:")
d3.json( $conf_data_filepath, function(d) {
    console.log(d)
})



function getType(num) {
	if(num == "0"){
		return "tech"
	}else if(num == "1"){
		return "business"
	}else if(num =="2"){
		return "politics"
	}else if(num == "3"){
		return "entertainment"
	}else if(num == "4"){
		return "sports"
	}else{
		return "invalid"
	}
}
    
d3.json("Raw_HTML_And_Data/example_test_result.json", function(d) {
        var table = new Array(5);
		var dataset = [];

		for(var i = 0; i < 5; i++){
			table[i] = new Array(5);
			for (var j = 0; j < 5; j++)    { 
				table[i][j] = 0
			} 
		}


        for (var i = 0; i < d.length; i += 1) {
			table[parseInt(d[i]["true_label"])][parseInt(d[i]["predicted_label"])] += 1;
			dataset.push([d[i]["true_label"],d[i]["predicted_label"],d[i]["text"],d[i]["index"]]);
        }


		var w = 1050;
        var h = 1050;
        
        //Create SVG element
        var svg = d3.select("body")
					.select("#matrix")
                    .append("svg")
                    .attr("width", w)
                    .attr("height", h);

					
		var rect = svg.selectAll("rect")
            .data(dataset)
            .enter()
            .append("rect")

		var counters = new Array(26).fill(0);
		var ycounters = new Array(26).fill(0);
		
        //d[0] = true_label, d[1] = predicted label, d[2] = text
        rect.attr("x", function(d, i) {
					var matrixnum = (parseInt(d[1])*5) + parseInt(d[0])
					var inmatrixcol = counters[matrixnum] % 12

					counters[matrixnum]++

					// margin +  which block + column within block
					return 10 + (d[0] * 220) + (inmatrixcol * 16);
                })
                .attr("y", function(d, i) {
					var matrixcol = d[1]
					var matrixnum = (parseInt(d[1])*5) + parseInt(d[0])
					var hm = Math.floor(ycounters[matrixnum]/12)

					ycounters[matrixnum]++

					// margin +  which block + column within block
					return 10 + (d[1] * 220) + (hm * 16);
                })
				.attr("id",function(d){
					return "rect" + d[3];
				})
                .attr("width", function(d) {
                    return 15;
                })
                .attr("height", function(d) {
                    return 15;
                })
                .attr("opacity", function(d) {
                    return .85;
                })
                .attr("fill", function(d) {
                    return ("pink");
                })
				.attr("class", function(d){
					predicted_label = "predicted_label_" + d[1];
					true_label = "true_label_" + d[0];

					return true_label + " " + predicted_label;
				})

	d3.select("#review")
		.select("#testList")
		.selectAll("rect")
			.data(
				dataset.filter(d => d[0] != d[1]),
				function(d){
					return d[3];
				}
			)
			.enter()
			.append("li")
			.attr("id",function(d){
				return "text" + d[3];
			})
			.html(function(d){
				table = "<table><tr>"

				table += "<td> True: ";
				table += getType(d[0]);
				table += "</td>"

				table += "<td> Predict: ";
				table += getType(d[1]);
				table += "</td>"

				table += "<td>" + d[2].substr(0,200); + "</td>"
				table += "</tr> </table>"
				return  table;
			})

    rect.on("click", function(d_on) {
		//always clear out the review to be blank on new click
		d3.select("#review")
					.select("#testList")
					.html("")
		//if its a new block being clicked set things back (purple)
		if(!this.classList.contains("past")){
			d3.selectAll(".past")
				.attr("fill","pink")
				.classed("past", false)
			d3.selectAll(".reclick")
				.attr("fill","pink")
				.classed("reclick", false)
		}
		//if its a new block being clicked set things back (yellow)
		if(!this.classList.contains("reclick")){
			d3.selectAll(".reclick")
				.attr("fill","pink")
				.classed("reclick", false)
		}

		d3.select(this)	
			//slack message
			textId = ""
			x = "." + this.classList[0];
			y = "."+ this.classList[1];
			test = x + y

			x1 = x.charAt(x.length-1)
			y1 = y.charAt(y.length-1)
			
			if(this.classList.contains("past")){
				d3.select(this)
					.classed("reclick",true)
				Id = this.id;
				textId = "#text" + Id.substring(4);
			}

			d3.selectAll(test)
				.attr("fill","purple")
				.classed("past", "true")

			d3.select("#review")
				.select("#testList")
				.selectAll("rect")
					.data(
						dataset
							.filter(d => d[0] == x1)
							.filter(d => d[1] == y1),
							function(d){
								return d[3];
							}
					)
					.enter()
					.append("li")
					.attr("id",function(d){
						return "text" + d[3];
					})
					.html(function(d){
						table = "<table><tr>"

						table += "<td> True: ";
						table += getType(d[0]);
						table += "</td>"

						table += "<td> Predict: ";
						table += getType(d[1]);
						table += "</td>"

						table += "<td>" + d[2].substr(0,200); + "</td>"
						table += "</tr> </table>"
						return  table;
					})
			d3.select(".reclick")
				.attr("fill","yellow")
			if(textId != ""){
				d3.selectAll(textId)
					.classed("highlight", true)
				
			}

		d3.select("#review")
			.select("#testList")
			.selectAll("li")
			.on("mouseover", function(d_on) {
				d3.select(this)	
					.classed("lighthigh",true)
					id = this.id;
					rectId = "#rect" + id.substring(4);
					d3.selectAll(rectId)	
						.attr("fill","green")
			})
			.on("mouseout", function(d_on) {
				d3.select(this)	
					.classed("lighthigh",false)
					id = this.id;
					rectId = "#rect" + id.substring(4);
					d3.selectAll(rectId)	
						.attr("fill", "purple")

			})

	})

	d3.select("#review")
		.select("#testList")
		.selectAll("li")
		.on("mouseover", function(d_on) {
			d3.select(this)	
				.classed("lighthigh",true)
				id = this.id;
				rectId = "#rect" + id.substring(4);
				d3.selectAll(rectId)	
					.attr("fill","green")

		})
		.on("mouseout", function(d_on) {
			d3.select(this)	
				.classed("lighthigh",false)
				id = this.id;
				rectId = "#rect" + id.substring(4);
				d3.selectAll(rectId)	
					.attr("fill","pink")


		})

	
});
    

    
    
''')

In [5]:
html_template = Template('''
<style> $css_text </style>
<div class="dispcont">
    <h1>Interactive Confusion Matrix</h1>
    <div id="matrix"> </div>
    <div id="review">
        Data
        <ul id = "testList">
        </ul>
    </div>
</div>
<script> $js_text </script>
''')

In [6]:
js_text = js_template.substitute({'confusion_data': json.dumps(local_json.to_json(orient="records")),
                                  'conf_data_filepath': local_json_filepath})

HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))